<a href="https://colab.research.google.com/github/heinrichhegenbarth/KAN-CDSCO2004U_Machine_Learning_and_Deep_Learning/blob/main/Ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

## Links
- https://cbscanvas.instructure.com/courses/41482/files/1504373?wrap=1
- https://insideairbnb.com/get-the-data/

## Approach
1. drop qualitative columns
### EDA
2. correlation analysis
3. graphing
### Clustering
4. choose clustering algorithm
5. run clustering algorithm
6. graph clusters
### PCA
7. run pca

In [1]:
# imports
import pandas as pd
import seaborn as sns
import sklearn 



In [2]:
# 1. drop qualitative columns
with open('./data/listings.csv', 'r') as file:
    df_listings = pd.read_csv(file)
    
with open('./data/reviews.csv', 'r') as file:
    df_reviews = pd.read_csv(file)


In [3]:
df_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,371299,Marais Rue des Archives refait à neuf février ...,1870265,Thomas,NaN,Hôtel-de-Ville,48.85751,2.35511,Entire home/apt,185.0,3,601,2024-09-06,3.97,3,307,54,7510300547558
1,371862,loft in Paris (Belleville),1872631,Veronique,NaN,Entrepôt,48.87151,2.37219,Entire home/apt,250.0,4,50,2023-08-14,0.34,1,9,0,7511000320406
2,372879,Appartement complet au centre de Paris.,1876396,Samuel,NaN,Gobelins,48.83593,2.35108,Entire home/apt,85.0,30,171,2024-08-15,2.01,3,151,1,"Available with a mobility lease only (""bail mo..."
3,375434,"Luxurious Art & Design Flat, 180m2, Champs Ely...",683140,Oscar,NaN,Élysée,48.86680,2.30972,Entire home/apt,NaN,3,22,2023-07-15,0.15,1,0,0,7510806561916
4,378897,Little flat near Montmartre,1902818,Lorraine,NaN,Opéra,48.88285,2.33462,Entire home/apt,110.0,3,28,2024-07-30,0.19,1,6,2,7511805895013


In [4]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3109,207127433,2017-10-28,51636494,Patricia,Tout s'est bien déroulé. Merci bien. PG
1,3109,208779822,2017-11-03,4142888,Patricia,Un petit nid fouiller douillet situé dans app...
2,3109,295840159,2018-07-24,7415343,Laurent,"Appartement spacieux, propre,clair, et calme à..."
3,3109,553502638,2019-10-24,21159216,Anastasia,"Appartement totalement rénové, en parfait état..."
4,5396,4824,2009-06-30,19995,Sarah,Perfect location!! Nasrine was a delight and m...


In [5]:
# data cleaning
## drop na prices
df_listings_clean = df_listings.dropna(subset=['price'])

## cleaning single columns
#def clean_price(price):
#    return float(price.replace('$', '').replace(',', ''))
#df_listings_clean['price'] = df_listings_clean['price'].apply(clean_price)

#def clean_perc(perc):
#    return float(perc.replace('%', ''))
#df_listings_clean['host_response_rate'] = df_listings_clean['host_response_rate'].apply(lambda x: clean_perc(x) if pd.notna(x) else x)
#df_listings_clean['host_acceptance_rate'] = df_listings_clean['host_acceptance_rate'].apply(lambda y: clean_perc(y) if pd.notna(y) else y)

#def is_superhost(superhost):
#    return 1 if superhost == 't' else 0
#df_listings_clean['host_is_superhost'] = df_listings_clean['host_is_superhost'].apply(lambda z: is_superhost(z) if pd.notna else z)

## irrelevant columns
drop_cols_listing = ['id', 'host_id', 'neighbourhood_group', 'license']
df_listings_clean = df_listings_clean.drop(columns=drop_cols_listing)


## non numeric columns
def drop_non_numeric(df):
    return df.select_dtypes(include=['number'])

df_listings_num = drop_non_numeric(df_listings_clean)
df_reviews_num = drop_non_numeric(df_reviews)


## fill with mean
#def fill_with_mean(df):
#    return df.fillna(df.mean())
#for col in df_listings_clean.columns:
#    df_listings_clean[col] = fill_with_mean(df_listings_clean[col])

df_listings_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64230 entries, 0 to 95460
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   name                            64230 non-null  object 
 1   host_name                       64230 non-null  object 
 2   neighbourhood                   64230 non-null  object 
 3   latitude                        64230 non-null  float64
 4   longitude                       64230 non-null  float64
 5   room_type                       64230 non-null  object 
 6   price                           64230 non-null  float64
 7   minimum_nights                  64230 non-null  int64  
 8   number_of_reviews               64230 non-null  int64  
 9   last_review                     46054 non-null  object 
 10  reviews_per_month               46054 non-null  float64
 11  calculated_host_listings_count  64230 non-null  int64  
 12  availability_365                64230

In [6]:
# 2. correlation analysis
## Convert column to list
for col in df_listings_num.columns:
    
    col_list = df_listings_num.loc[:, col].dropna().tolist()
## Calculate mean
    sum_col = sum(col_list)
    count_col = len(col_list)
    mean_col = round(sum_col / count_col,6)
        
## Calculate variance
    squared_dist = sum([(mean_col-x)**2 for x in col_list])
    var_col = squared_dist / count_col

## Calculate standard deviation  
    sd_col = var_col ** (1/2)

## print results
    print(f"Mean of {col} is {mean_col}")
    print(f"Variance of {col} is {var_col}")
    print(f"Standard Deviation of {col} is {sd_col}")

Mean of latitude is 48.863744
Variance of latitude is 0.00032427193946897137
Standard Deviation of latitude is 0.01800755228977474
Mean of longitude is 2.341303
Variance of longitude is 0.0011955562091951308
Standard Deviation of longitude is 0.03457681606503309
Mean of price is 256.023276
Variance of price is 272763.9950210616
Standard Deviation of price is 522.2681256031825
Mean of minimum_nights is 10.628429
Variance of minimum_nights is 1632.1196339482249
Standard Deviation of minimum_nights is 40.3995004170624
Mean of number_of_reviews is 25.267601
Variance of number_of_reviews is 3739.2085704076717
Standard Deviation of number_of_reviews is 61.14906843450415
Mean of reviews_per_month is 1.35002
Variance of reviews_per_month is 2.0355042923789815
Standard Deviation of reviews_per_month is 1.4267110052070746
Mean of calculated_host_listings_count is 31.053884
Variance of calculated_host_listings_count is 11569.664183494051
Standard Deviation of calculated_host_listings_count is 107

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=len(df_listings_clean.columns), ncols=1, figsize=(10, 5 * len(df_listings_clean.columns)))

for ax, col in zip(axes, df_listings_clean.columns):
    sns.scatterplot(data=df_listings_clean, y='price', x=col, ax=ax)
    ax.set_title(col)

plt.tight_layout()
plt.show()
    

KeyboardInterrupt: 